In [ ]:
#Generate Energy List
import forgi.threedee.model.coarse_grain as ftmc
import os

def get_rmsd_dict(rmsd_list):
    rmsd_dict = {}
    with open(rmsd_list, "r") as fh:
        for line in fh.readlines():
            name, rmsd = (line.rstrip()).split("\t")
            rmsd_dict[name] = float(rmsd)
    return rmsd_dict

rmsd_dict = get_rmsd_dict("../data/train_rmsd_list.txt")

file_path = "../data/train_set/" #"../data/old_training_set/" # "../data/val_set/"#"../data/test_set/" #

energy_file = "../data/train_energy.txt" #"../data/old_train_energy.txt"#"../data/val_energy.txt"#"../data/test_energy.txt" #

write = False

#file = "../data/train_set/1c2x_C_5srRNA_C_br0.cg"
c_files = 0
c_has = 0
c_not = 0
pos_e = 0
lines = []
for file in [f for f in os.listdir(file_path) if os.path.isfile(os.path.join(file_path, f))]:
    if file in rmsd_dict.keys():
        cg = ftmc.CoarseGrainRNA.from_bg_file(file_path + file)
        cg.radius_of_gyration
        energy = cg.infos['Energy']
    
        c_files += 1
        if energy == []:
            c_not += 1
            print(file, energy, cg.radius_of_gyration(method="vres"), rmsd_dict[file])
        elif "ROG" in energy[0]:
            c_has +=1
            rog_en = energy[0].split(",")[0][:-4]
            if float(rog_en) > 0:
                #print(file, rog_en)
                pos_e +=1
            line = file + "\t" + rog_en + "\n"
            lines.append(line)

    

print("files", c_files)
print("ROG energy", c_has)
print("Positive energy", pos_e)
print("no energy", c_not)
print(c_has + c_not)

if write:
    with open(energy_file, "w") as fh:
        for l in lines:
            fh.write(l)


In [23]:
#Rotate cg structures
import forgi.threedee.model.coarse_grain as ftmc
import forgi.threedee.utilities.vector as ftuv
import forgi.utilities.commandline_utils as fuc
import os

directory = "../data/test_set/"
out_dir = "../data/test_set_rot/"
angle = 1.5708


for file in [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]:


    cgf = ftmc.CoarseGrainRNA.from_bg_file(directory + file)

    cgf.rotate(angle)



    with fuc.open_for_out(out_dir + file) as outfile:
        print(cgf.to_cg_string(), file=outfile)

In [6]:
#Transform coords to vectors, pointing to the next connected element
import forgi.threedee.model.coarse_grain as ftmc
import forgi.threedee.utilities.vector as ftuv
import forgi.utilities.commandline_utils as fuc
import os
import numpy as np

cg = ftmc.CoarseGrainRNA.from_bg_file("../data/train_set/1c2x_C_5srRNA_C_be10.cg")

cg_d = dict(cg.coords)

#for e in cg_d:
#    print(e, cg_d[e])

print(cg.coords.get_direction("t0"))

dist = cg.element_physical_distance("t0", "m0")

print(dist)
for elem in cg_d:
    vector = []
    if elem == "t0" or elem == "f0":
        for i in range(3):
            a = cg_d[elem][0][i] - cg_d[elem][1][i]
            vector.append(a)
    else:
        for i in range(3):
            a = cg_d[elem][1][i] - cg_d[elem][0][i]
            vector.append(a)

    print(elem, np.array(vector))

[ 9.6719918  -3.48100864 -1.15317169]
17.849
s0 [ 0.    17.849  0.   ]
m0 [ 9.1820551  10.94019783  3.16790342]
s1 [-0.06612564 -2.56114239  0.16020625]
i0 [ 0.40567371 -5.40125577  1.58513772]
s2 [ 0.67507931 -5.06854354  0.08444098]
i1 [ 3.56446688  6.38357025 12.75647898]
s3 [-6.01970619 -1.85064955  4.36224193]
h0 [-5.1620225  25.22911748 -9.45248275]
m1 [ 1.15852993  5.37241537 -0.38377324]
s4 [ 1.02590916  2.29657981 -0.51256276]
i2 [-1.72501499 17.34190355  8.31710371]
s5 [ 7.17459534  3.50431681 15.96345033]
h1 [ 2.89447963 -8.16404602  9.34800693]
m2 [-10.34058503 -16.31261319  -2.78413018]
t0 [-9.6719918   3.48100864  1.15317169]


In [1]:
#get nearest k elements
#--> set of k vectors
#plus origin? ie middle of element
import forgi.threedee.model.coarse_grain as ftmc
import forgi.threedee.utilities.vector as ftuv
import forgi.utilities.commandline_utils as fuc
import os
import numpy as np

cg = ftmc.CoarseGrainRNA.from_bg_file("../data/train_set/1c2x_C_5srRNA_C_be10.cg")

cg_d = dict(cg.coords)

#calculate midpoint of each element
mp_dir = {}
for elem in cg_d:
    mp = (cg_d[elem][0] + cg_d[elem][1])/2
    mp_dir[elem] = mp

for e in mp_dir:
    print(e, mp_dir[e])

#calculate distance from each midpoint to every other
dist_dir = {}
for a in mp_dir:
    helper_d = {}
    for b in mp_dir:
        if a != b:
            dist = np.linalg.norm(mp_dir[b] - mp_dir[a])
            helper_d[b] = dist
    if helper_d != {}:
        dist_dir[a] = helper_d

#get the nearest k=5 elements
n_dict = {}
for f in dist_dir:
    n_list = []
    #print(f, {k: v for k, v in sorted(dist_dir[f].items(), key=lambda item: item[1])})
    i = 0
    for n in  {k: v for k, v in sorted(dist_dir[f].items(), key=lambda item: item[1])}:
        n_list.append(n)
        i+=1
        if i == 3:
            break
    n_dict[f] = n_list

print(n_dict)

for elem in n_dict:
    v_arr = []
    for e in n_dict[elem]:
        v_arr.append(mp_dir[e])
    print(elem, np.concatenate(v_arr))

s0 [0.     8.9245 1.    ]
m0 [ 4.59102755 23.31909891  2.58395171]
s1 [ 9.14899228 27.50862663  4.24800654]
i0 [ 9.31876632 23.52742755  5.12067852]
s2 [ 9.85914283 18.2925279   5.95546788]
i1 [11.97891593 18.95004125 12.37592786]
s3 [10.75129627 21.21650161 20.93528831]
h0 [ 5.16043193 32.90573557 18.3901679 ]
m1 [ 9.76132007 31.47540551  3.9760168 ]
s4 [10.85353961 35.3099031   3.5278488 ]
i2 [10.50398669 45.12914478  7.43011927]
s5 [13.22877687 55.55225496 19.57039629]
h1 [18.26331436 53.22239035 32.22612492]
m2 [ 5.17029252 26.0053066   2.39206509]
t0 [ 4.8359959  -1.74050432  0.42341416]
{'s0': ['t0', 's2', 'm0'], 'm0': ['m2', 'i0', 's1'], 's1': ['m1', 'i0', 'm2'], 'i0': ['s1', 's2', 'm0'], 's2': ['i0', 'i1', 'm0'], 'i1': ['s2', 's3', 'i0'], 's3': ['i1', 'h0', 's2'], 'h0': ['s3', 'm1', 's1'], 'm1': ['s4', 's1', 'm2'], 's4': ['m1', 's1', 'i2'], 'i2': ['s4', 'm1', 's5'], 's5': ['h1', 'i2', 'h0'], 'h1': ['s5', 'i2', 'h0'], 'm2': ['m0', 's1', 'i0'], 't0': ['s0', 's2', 'i1']}
s0 [ 4.83

In [6]:
#Open lmdb files
import lmdb
import gzip
import json
import numpy as np
import pandas as pd
#import pdbtools

'''
b'id_to_idx'
b'num_examples'
b'serialization_format'
'''


lmdb_file = "/home/milan/Programs/ares_release/data/lmdbs/train/"
lmdb_env = lmdb.open(lmdb_file)
lmdb_txn = lmdb_env.begin()
lmdb_cursor = lmdb_txn.cursor()

print(lmdb_env.stat())

idx = lmdb_cursor.get("idx_to_idx".encode())#.decode() #gzip.decompress(
print(idx)


i = gzip.decompress(lmdb_cursor.get("0".encode())).decode()

d = json.loads(i)

i_json = pd.read_json(i)

columns = i_json.iloc[0,0]
index = i_json.iloc[1,0]
data = i_json.iloc[2,0]

i_new = pd.DataFrame(data=data, index=index, columns=columns)

pdb_out = pd.DataFrame(i_new, columns=["serial_number", "subunit", "chain", "residue", "resname", "x", "y", "z", "element", "name"])

pdb_out.head()

{'psize': 4096, 'depth': 1, 'branch_pages': 0, 'leaf_pages': 1, 'overflow_pages': 256, 'entries': 53}
None


,serial_number,subunit,chain,residue,resname,x,y,z,element,name
0,1,0,A,1,C,7.278,9.150,14.138,C,C5'
1,2,0,A,1,C,6.457,10.282,13.572,C,C4'
2,3,0,A,1,C,7.273,11.210,12.812,O,O4'
3,4,0,A,1,C,5.459,9.762,12.567,C,C3'
4,5,0,A,1,C,4.329,9.250,13.226,O,O3'


In [25]:
#get list of representative RNA structures

rep_pdbs = []
with open("../data/nrlist_3.220_4.0A.csv", "r") as fh:
    for line in fh.readlines():
        all = line.split(",")[1].lower()
        pdb = all.lstrip('"').split("|")[0]
        rep_pdbs.append(pdb)

rfam_pdbs = []
with open("../data/Rfam.pdb", "r") as fh:
    for line in fh.readlines()[1:]:
        pdb = line.split("\t")[1]
        rfam_pdbs.append(pdb)


c = 0
for item in rep_pdbs:
    if item not in rfam_pdbs:
        c+=1

print("len rep pdbs", len(rep_pdbs))
print("len rfam pdbs", len(rfam_pdbs))
print("rep not in rfam", c)


len rep pdbs 2312
len rfam pdbs 7458
rep not in rfam 1720


In [36]:
rep_pdbs = []
with open("../data/nrlist_3.220_4.0A.csv", "r") as fh:
    for line in fh.readlines():
        all = line.split(",")[1].lower().lstrip('"').rstrip('"')
        if "+" in all:
            l = all.split("+")
            for i in l:
                id = i.split("|")
                rep_pdbs.append(id)
        else:
            pdb = all.split("|")
            rep_pdbs.append(pdb)

print(len(rep_pdbs))
with open("../data/reprna_list.txt", "w") as fh:
    for n in rep_pdbs:
        fh.write("\t".join(n) + "\n")

2862


In [12]:
from pdbecif.mmcif_io import CifFileReader

cfr = CifFileReader()
cif_obj = cfr.read("/home/milan/Downloads/6zmi.cif", output='cif_wrapper')
#print(cif_obj)

cif_data = list(cif_obj.values())[0]

#print(cif_data)


#print(cif_data._entity_poly_seq)

print(cif_data._entity_poly.structure)


#print(cif_data['_entity_poly']['pdbx_strand_id'])


#print(cif_data._entity_poly.search('entity_id', '1'))

print(cif_data._entity_poly.search('pdbx_strand_id', 'L5'))

None
{0: {'entity_id': '1', 'type': 'polyribonucleotide', 'nstd_linkage': 'no', 'nstd_monomer': 'no', 'pdbx_seq_one_letter_code': 'CGCGACCUCAGAUCAGACGUGGCGACCCGCUGAAUUUAAGCAUAUUAGUCAGCGGAGGAGAAGAAACUAACCAGGAUUCC\nCUCAGUAACGGCGAGUGAACAGGGAAGAGCCCAGCGCCGAAUCCCCGCCCCGCGGCGGGGCGCGGGACAUGUGGCGUACG\nGAAGACCCGCUCCCCGGCGCCGCUCGUGGGGGGCCCAAGUCCUUCUGAUCGAGGCCCAGCCCGUGGACGGUGUGAGGCCG\nGUAGCGGCCCCCGGCGCGCCGGGCCCGGGUCUUCCCGGAGUCGGGUUGCUUGGGAAUGCAGCCCAAAGCGGGUGGUAAAC\nUCCAUCUAAGGCUAAAUACCGGCACGAGACCGAUAGUCAACAAGUACCGUAAGGGAAAGUUGAAAAGAACUUUGAAGAGA\nGAGUUCAAGAGGGCGUGAAACCGUUAAGAGGUAAACGGGUGGGGUCCGCGCAGUCCGCCCGGAGGAUUCAACCCGGCGGC\nGGGUCCGGCCGUGUCGGCGGCCCGGCGGAUCUUUCCCGCCCCCCGUUCCUCCCGACCCCUCCACCCGCCCUCCCUUCCCC\nCGCCGCCCCUCCUCCUCCUCCCCGGAGGGGGCGGGCUCCGGCGGGUGCGGGGGUGGGCGGGCGGGGCCGGGGGUGGGGUC\nGGCGGGGGACCGUCCCCCGACCGGCGACCGGCCGCCGCCGGGCGCAUUUCCACCGCGGCGGUGCGCCGCGACCGGCUCCG\nGGACGGCUGGGAAGGCCCGGCGGGGAAGGUGGCUCGGGGGGCCCCGUCCGUCCGUCCGUCCGUCCUCCUCCUCCCCCGUC\nUCCGCCCCCCGGCCCCGCGUCCUCCCUCGGGAGGGCGCGCGGGUCGGGG

In [18]:
#from: https://stackoverflow.com/questions/11685716/how-to-extract-chains-from-a-pdb-file
#modify to get selected chains
import os
from Bio import PDB


class ChainSplitter:
    def __init__(self, out_dir=None):
        """ Create parsing and writing objects, specify output directory. """
        self.parser = PDB.PDBParser()
        self.writer = PDB.PDBIO()
        if out_dir is None:
            out_dir = os.path.join(os.getcwd(), "chain_PDBs")
        self.out_dir = out_dir

    def make_pdb(self, pdb_path, chain_letters, overwrite=False, struct=None):
        """ Create a new PDB file containing only the specified chains.

        Returns the path to the created file.

        :param pdb_path: full path to the crystal structure
        :param chain_letters: iterable of chain characters (case insensitive)
        :param overwrite: write over the output file if it exists
        """
        #chain_letters = [chain.upper() for chain in chain_letters]

        # Input/output files
        (pdb_dir, pdb_fn) = os.path.split(pdb_path)
        pdb_id = pdb_fn[3:7]
        out_name = "pdb%s_%s.ent" % (pdb_id, "".join(chain_letters))
        out_path = os.path.join(self.out_dir, out_name)
        print("OUT PATH:",out_path)
        plural = "s" if (len(chain_letters) > 1) else ""  # for printing

        # Skip PDB generation if the file already exists
        if (not overwrite) and (os.path.isfile(out_path)):
            print("Chain%s %s of '%s' already extracted to '%s'." %
                    (plural, ", ".join(chain_letters), pdb_id, out_name))
            return out_path

        print("Extracting chain%s %s from %s..." % (plural,
                ", ".join(chain_letters), pdb_fn))

        # Get structure, write new file with only given chains
        if struct is None:
            struct = self.parser.get_structure(pdb_id, pdb_path)
        self.writer.set_structure(struct)
        self.writer.save(out_path, select=SelectChains(chain_letters))

        return out_path


class SelectChains(PDB.Select):
    """ Only accept the specified chains when saving. """
    def __init__(self, chain_letters):
        self.chain_letters = chain_letters

    def accept_chain(self, chain):
        return (chain.get_id() in self.chain_letters)

if __name__ == "__main__":
    """ Parses PDB id's desired chains, and creates new PDB structures. """
    import sys
    #if not len(sys.argv) == 2:
    #    print("Usage: $ python %s 'pdb.txt'" % __file__)
    #    sys.exit()

    pdb_textfn = "/home/milan/testpdb.txt" #sys.argv[1]

    pdbList = PDB.PDBList()
    splitter = ChainSplitter("/home/milan/MS_Arbeit/RepRNA_PDBs")  # Change me.

    with open(pdb_textfn) as pdb_textfile:
        for line in pdb_textfile:
            pdb_id, el, chain = (line.lower()).split("\t")
            print(pdb_id, el, chain)
            #chain = line[4:]
            #pdb_fn = pdbList.retrieve_pdb_file(pdb_id)
            #splitter.make_pdb(pdb_fn, chain)

4v9q 1 dv

